In [ ]:
import os
import librosa
import csv
import torch
import numpy as np

from PIL import Image
# import requests
from transformers import AutoProcessor, CLIPVisionModelWithProjection

# jpg_dir =  "/Users/manshisagar/Desktop/sem-7 assign/col865/project/COL865_MMRS/ml-100k/Image"
# ekansh
jpg_dir = "/Users/manshisagar/Desktop/sem-7assign/col865/project/COL865_MMRS/movie_posters_resized"

model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

# def resize_and_preprocess_image(image_path, target_size=(256, 256)):
#     image = Image.open(image_path)
#     image = image.resize(target_size)
#     inputs = processor(images=image, return_tensors="pt")
#     return inputs

image_files = [filename for filename in os.listdir(jpg_dir) if filename.lower().endswith(('.jpg', '.png', '.jpeg'))]
print(len(image_files))



arr_image_embeds= []

# Loop through all the image files and process them
for i in range(800):
    # print(filename)
    filename = image_files[i]
    print(i)
    file_path = os.path.join(jpg_dir, filename)
    image_sample = Image.open(file_path)
    inputs = processor(images=image_sample, return_tensors="pt")
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    arr_image_embeds.append(image_embeds)


In [ ]:
for i in range(800, len(image_files)):
    # print(filename)
    filename = image_files[i]
    print(i)
    file_path = os.path.join(jpg_dir, filename)
    image_sample = Image.open(file_path)
    inputs = processor(images=image_sample, return_tensors="pt")
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    arr_image_embeds.append(image_embeds)

# for i in range(len(image_files)):
#     filename = image_files[i]
#     print(i)
#     file_path = os.path.join(jpg_dir, filename)
#     inputs = resize_and_preprocess_image(file_path)
#     outputs = model(**inputs)
#     image_embeds = outputs.image_embeds
#     arr_image_embeds.append(image_embeds)

print("size of tensor == ", len(arr_image_embeds[0]))
print("size of arr_image_embeds== ", len(arr_image_embeds))
new_image_embeds= []
for t in arr_image_embeds:
    temp= t.squeeze()
    new_image_embeds.append(temp)

print("size of tensor == ", len(new_image_embeds[0]))
print("size of new_image_embeds== ", len(new_image_embeds))
resized_image_embeds= []
# resizing text embeddings to 384 
for t in new_image_embeds:
    resized_i = torch.nn.functional.interpolate(t.unsqueeze(0).unsqueeze(0), size = (4096,), mode = 'linear').squeeze()
    resized_image_embeds.append(resized_i)

print("size of new tensor == ", resized_image_embeds[0].size())
print("size of resized_image_embeds== ", len(resized_image_embeds))
# saving text embeddings in text_feat.npy
numpy_array = [tensor.detach().numpy() for tensor in resized_image_embeds]

stacked_resized_text_embeds = np.stack(numpy_array, axis=0)

np.save('images_feat.npy',stacked_resized_text_embeds)
print("saved text embeddings in image_feat.npy")